In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import mnist
from NeutralNet import NeuralNet
from MyModule import *
import os

# 判定GPU是否存在

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 定义超参数
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

# 数据集dir
data_dir = 'resources/dataset'

In [2]:
# 载入手写体数据 如果不存在则下载 注意科学上网，否则下载很慢 
train_dataset = torchvision.datasets.MNIST(root=data_dir,
                                           train=True,
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root=data_dir,
                                          train=False,
                                          transform=transforms.ToTensor())
print("download dataset successfully!")

download dataset successfully!


In [3]:
# 构建数据管道
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

# 定义模型
# model = NeuralNet(input_size, hidden_size, num_classes).to(device)
model = In300OutNet(activation='relu').to(device)
# 损失函数和优化算法
# criterion = nn.CrossEntropyLoss()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# , weight_decay=1e-8
# 初始化权重和偏置
# for name,param in model.named_parameters():
#         if 'weight' in name:
#             nn.init.kaiming_uniform_(param, nonlinearity='relu')
#         if 'bias' in name:
#             nn.init.zeros_(param)

print(next(model.parameters()).is_cuda)
print("build model successfully!")

True
build model successfully!


In [4]:
# 训练模型
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        labels = mnist.labels_to_one_hot(labels=labels)
        # Move tensors to the configured device
        images = images.reshape(-1, 28 * 28).to(device)
        labels = labels.to(device)
        # 前向传播和计算loss
        outputs = model(images)
        loss = criterion(outputs, labels)

        # 后向传播和调整参数
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # 每100个batch打印一次数据
        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.0272
Epoch [1/5], Step [200/600], Loss: 0.0220
Epoch [1/5], Step [300/600], Loss: 0.0211
Epoch [1/5], Step [400/600], Loss: 0.0156
Epoch [1/5], Step [500/600], Loss: 0.0178
Epoch [1/5], Step [600/600], Loss: 0.0131
Epoch [2/5], Step [100/600], Loss: 0.0094
Epoch [2/5], Step [200/600], Loss: 0.0164
Epoch [2/5], Step [300/600], Loss: 0.0112
Epoch [2/5], Step [400/600], Loss: 0.0121
Epoch [2/5], Step [500/600], Loss: 0.0127
Epoch [2/5], Step [600/600], Loss: 0.0084
Epoch [3/5], Step [100/600], Loss: 0.0081
Epoch [3/5], Step [200/600], Loss: 0.0110
Epoch [3/5], Step [300/600], Loss: 0.0087
Epoch [3/5], Step [400/600], Loss: 0.0099
Epoch [3/5], Step [500/600], Loss: 0.0074
Epoch [3/5], Step [600/600], Loss: 0.0079
Epoch [4/5], Step [100/600], Loss: 0.0095
Epoch [4/5], Step [200/600], Loss: 0.0072
Epoch [4/5], Step [300/600], Loss: 0.0059
Epoch [4/5], Step [400/600], Loss: 0.0089
Epoch [4/5], Step [500/600], Loss: 0.0127
Epoch [4/5], Step [600/600], Loss:

In [5]:
# 模型测试部分
# 测试阶段不需要计算梯度，注意
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28 * 28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        # print(_, predicted)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

Accuracy of the network on the 10000 test images: 97.8 %


In [6]:
flag = torch.cuda.is_available()
device_cnt = torch.cuda.device_count()
device_name=torch.cuda.get_device_name(0)
device_curr = torch.cuda.current_device()

print(flag)
print(device_cnt)
print(device_name)
print(device_curr)

True
1
GeForce GTX 1050 Ti
0
